In [393]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib, os, fiona
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
from shapely.ops import nearest_points, polygonize, linemerge
import Outils
#pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 50)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# IMPORTER LES DONNEES 
> depuis la base de données locale, en ne prenant en compte que les lignes qui intersectent une zone carré au N-O

In [140]:
df=at.import_donnes_base('local_otv')

>Mise en forme et définition des voies à 2 chaussées

In [141]:
nature_2_chaussees=['Route à 2 chaussées','Quasi-autoroute','Autoroute']
#donnees generale du jdd
#df.set_index('id_ign', inplace=True)  # passer l'id_ign commme index ; necessaire sinon pb avec geometrie vu comme texte
df2_chaussees=df.loc[df['nature'].isin(nature_2_chaussees)] #isoler les troncon de voies decrits par 2 lignes

# 1. BDD DES RONDS POINTS

In [142]:
df_avec_rd_pt, carac_rd_pt=at.identifier_rd_pt(df)

# 2. TRAITEMENTS 
> La base c'est : 
- regrouper les lignes qui se suivent
- trouver les lignes d'arret
- trouver les lignes suivantes
- voir le cas des lignes suivantes : 
    - rd point
    - mm route qui se sépare
    - rien

><br> L'idée c'est ça  : on met la premiere ligne a tester dans une liste, on l'utilise et on stocke les lignes associées dans une autre liste. ensuite on check les lignes suivantes : si elles sont dans un cas où il faut continuer les traitements, on remplace la ligne de départ dans la liste de départ par ces nouvelles lignes, pour lesquelles on applique de nouveau la recherche des lignes qui se suivent. tant que la liste des lignes suivantes est pa vide on reste dans une boucle while


## 2.1 Troncons de base

In [496]:
# ligne a tester : TRONROUT0000000339411609 TRONROUT0000000032869130 TRONROUT0000000032859410
df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index
id_lignes='TRONROUT0000000032837545'
#trouver tous les troncons proches qui se suivent
#list_troncon=[id_ligne]+[a for a in at.liste_troncon_base(id_ligne,df_lignes)]
list_troncon=at.liste_complete_tronc_base(id_lignes,df_lignes,[])

## 2.2 lignes d'arret

In [430]:
#trouver les troncons de debut et de fin :ceux pour lesquels on a un nombre de ligne sur target source >2
dico_deb_fin=at.deb_fin_liste_tronc_base(df_lignes, list_troncon)

In [431]:
dico_deb_fin

{'TRONROUT0000000202010499': {'type': 'target',
  'num_node': 60815,
  'voie': 'NC',
  'codevoie': '171420392'},
 'TRONROUT0000000032842521': {'type': 'source',
  'num_node': 732,
  'voie': 'NC',
  'codevoie': '171420392'}}

In [432]:
ligne='TRONROUT0000000202010499'
voie=dico_deb_fin[ligne]['voie']
noeud=dico_deb_fin[ligne]['num_node']
codevoie=dico_deb_fin[ligne]['codevoie']

## 2.3 lignes suivantes

In [433]:
#vérifier si une ligne touche un rd point
# aprtir d'une ligne et d'un numero de noeud, on récupère les lignes qui touchent
lignes_adj=df_lignes.loc[((df_lignes['source']==noeud)|(df_lignes['target']==noeud))&(df_lignes.index!='TRONROUT0000000202010499')]

In [434]:
lignes_adj

,geom,id,nature,numero,importance,cl_admin,gestion,mise_serv,fictif,franchisst,largeur,z_ini,z_fin,nb_voies,sens,etat,long_km,coment_cpt,src_cpt,id_comptag,obs_supl,ann_pt,coment_tmj,tmja,tmja_final,...,vts_osm,vts_modif,vts_vl_f,vts_pl_f,codau,codau_cont,tmja_cont,pc_pl_cont,source,target,recup,attr_modif,codevoie_d,id_tronc_elem,nb_intrsct_src,src_geom,nb_intrsct_tgt,tgt_geom,id_rdpt,nb_rte_rdpt,nom_rte_rdpt,codevoie_rdpt,nb_obj_sig_entrant,valeur_sens,id_ign_entrant
id_ign,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TRONROUT0000000032842576,(LINESTRING (386701.5999999997 6572980.8999999...,17183,Route à 1 chaussée,NC,5,Autre,NC,NR,Non,NC,3.0,25.3,24.8,1.0,Inverse,NR,0.017320,estimation,None,None,None,None,/2,600.0,600.0,...,None,None,46.0,40.0,111+,None,None,None,60815,29626,t,None,171420392,NaN,3,POINT(386701.6 6572980.89999999),3,POINT(386684.3 6572980.49999999),NaN,NaN,NaN,NaN,NaN,NaN,NaN
TRONROUT0000000202010498,(LINESTRING (386701.5999999997 6572980.8999999...,216006,Route à 1 chaussée,NC,5,Autre,NC,NR,Non,NC,3.0,25.3,25.4,1.0,Direct,NR,0.017576,estimation,None,None,None,None,/2,600.0,600.0,...,None,None,46.0,40.0,111+,None,None,None,60815,30386,t,None,171420392,NaN,3,POINT(386701.6 6572980.89999999),3,POINT(386686.6 6572989.99999999),NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.4 cas particulier 

> ### 2.4.1 rd pt

In [237]:
#verfier si une ligne rouche un rd point et recup le numero si c'est le cas
check_rdpt, num_rdpt=at.verif_touche_rdpt(lignes_adj)
if check_rdpt : 
    lignes_rdpt, lignes_sortantes=at.recup_lignes_rdpt(carac_rd_pt,num_rdpt,list_troncon,'NC')

> ### 2.4.2 mm route qui se sépare

In [435]:
#comparer le nom de voie de la ligne issue de debut_fin avec celui des lignes adjacentes
if (voie==lignes_adj.numero).all() and voie!='NC' : 
    ligne_mm_voie=lignes_adj.index.tolist()
    list_troncon+=ligne_mm_voie
    lignes_suivantes_mm_nom=ligne_mm_voie #on recommence les traitements avec les lignes prises
elif voie=='NC' and codevoie!='NR' and (codevoie==lignes_adj.codevoie_d).all(): 
    ligne_mm_voie=lignes_adj.index.tolist()
    list_troncon+=ligne_mm_voie
    lignes_suivantes_mm_nom=ligne_mm_voie

In [436]:
ligne_mm_voie

['TRONROUT0000000032842576', 'TRONROUT0000000202010498']

> ### 2.4.2 trouver les routes coinces entre 2 voies qui se sépare -<|
>> ce sont des lignes qui  dont : 
- le debut et / ou la fin touche 3 lignes
- les lignes qui les touchent se toucehnt aussi

In [61]:
# trouver les lgnes qui en touchent 3 ou plus au debut et à la fin
lignes_sup3_tch=df_lignes.loc[(df_lignes['nb_intrsct_src']>2)&(df_lignes['nb_intrsct_tgt']>2)].reset_index()[['id_ign', 'source','target']]

In [58]:
%%time
#coupler les lignes avec celles qui se touchent
def ligne_touchent(source, target, df_lignes, id_ign) : 
    lignes=df_lignes.loc[(df_lignes.source.isin([source, target]))|(df_lignes.target.isin([source, target]))
                        &(df_lignes['id_ign']!=id_ign)]
    return lignes.id_ign.tolist()
lignes_sup3_tch['autre_ligne_tch']=lignes_sup3_tch.apply(lambda x : ligne_touchent(x['source'], x['target'],
                                                         lignes_sup3_tch, x['id_ign']), axis=1)

Wall time: 38.5 s


In [ ]:
liste_lgn_src=lignes_sup3_tch.groupby('source').agg({'id_ign':lambda x : tuple(x)})
liste_lgn_tgt=lignes_sup3_tch.groupby('target').agg({'id_ign':lambda x : tuple(x)})
lignes_sup3_tch.merge(liste_lgn_src, left_on='source', right_index=True).merge(
left_on='target')

## 2.5 route à 2 chaussées
L'idée c'est si la ligne est une route à 2 chaussée : 
- on prend l'ensemble du troncon elementaire
- on agerge les lignes
- on recupere le centre
- on recupere la longueur
- on pourrait chercher à comparer les bouding box

In [506]:
if id_lignes in df2_chaussees.id_ign.tolist() : 
    lgn_tron_e=df_avec_rd_pt.loc[df_avec_rd_pt['id_ign'].isin(list_troncon)]
    try : #union des geometries, si possible en linestring sinon en multi
        lgn_agrege=linemerge(lgn_tron_e.unary_union) #union des geometries
    except ValueError :
        lgn_agrege=lgn_tron_e.unary_union
    longueur_base=lgn_agrege.length
    xmin,ymin,xmax,ymax=lgn_agrege.interpolate(0.5, normalized=True).buffer(50).bounds #limtes du carre englobant du buffer 50 m du centroid de la ligne
    #gdf_global=gp.GeoDataFrame(df, geometry='geom')#donnees de base
    lignes_possibles=df_avec_rd_pt.cx[xmin:xmax,ymin:ymax]#recherche des lignes proches du centroid
    #uniquement les lignes non présentes dans la liste de troncons avec le même nom de voie
    ligne_filtres=lignes_possibles.loc[(~lignes_possibles.id_ign.isin(list_troncon)) & (lignes_possibles.loc[:,'numero']==lgn_tron_e.iloc[0]['numero'])].copy()
    #obtenir les distances au centroid
    ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : lgn_agrege.centroid.distance(x))
    #garder uniquement la valeur la plus proche du centroid
    ligne_proche=ligne_filtres.loc[ligne_filtres['distance']==ligne_filtres['distance'].min()].id_ign.tolist()[0]

In [505]:
ligne_proche

'TRONROUT0000000032837542'

In [508]:
list_troncon_comp=at.liste_complete_tronc_base(ligne_proche,df_lignes,[])

In [512]:
lgn_tron_comp=df_avec_rd_pt.loc[df_avec_rd_pt['id_ign'].isin(list_troncon_comp)]
try : #union des geometries, si possible en linestring sinon en multi
    lgn_agrege_comp=linemerge(lgn_tron_comp.unary_union) #union des geometries
except ValueError :
    lgn_agrege_comp=lgn_tron_comp.unary_union
long_comp=lgn_agrege_comp.length

In [513]:
long_comp

3288.7396002227765

In [507]:
longueur_base

2701.29217640315

# Tests de deroulement global

In [473]:
def lignes_troncon_elem(df_avec_rd_pt, ligne) : 
    """
    trouver les lignes qui appartiennent au même troncon elementaire que la ligne de depart
    en entree : 
        df_avec_rd_pt : df des lignes Bdtopo
        ligne : string : id_ign de la ligne a tester
    en sortie :
        liste_troncon_finale : set de string des id_ign des lignes réunies dans le troncon elementaire
    """
    df_lignes2=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index
    lignes_a_tester, liste_troncon_finale, list_troncon2=[ligne],[],[]

    while lignes_a_tester :
        for id_lignes2 in lignes_a_tester : 
            list_troncon2=at.liste_complete_tronc_base(id_lignes2,df_lignes2,liste_troncon_finale)
            liste_troncon_finale+=list_troncon2
            dico_deb_fin2=at.deb_fin_liste_tronc_base(df_lignes2, list_troncon2)
            for k, v in dico_deb_fin2.items() :
                lignes_adj2=df_lignes2.loc[((df_lignes2['source']==v['num_node'])|
                                          (df_lignes2['target']==v['num_node']))&
                                         (df_lignes2.index!=k)&(~df_lignes2.index.isin(liste_troncon_finale))]
                print(k, (v['codevoie']==lignes_adj2.codevoie_d).all(),lignes_adj2.codevoie_d.tolist(), v['codevoie'],at.recup_route_split(v['voie'],v['codevoie'], lignes_adj2) )
                if lignes_adj2.empty : 
                    continue
                check_rdpt2, num_rdpt2=at.verif_touche_rdpt(lignes_adj2)
                # on attaque la liste des cas possible
                #1. Rond point
                if check_rdpt2 : 
                    lignes_rdpt2, lignes_sortantes2=at.recup_lignes_rdpt(carac_rd_pt,num_rdpt2,list_troncon2,v['voie'])
                    liste_troncon_finale+=lignes_rdpt2
                    lignes_a_tester+=lignes_sortantes2
                #2. route qui se sépare
                elif at.recup_route_split(v['voie'],v['codevoie'], lignes_adj2) : 
                    liste_rte_separe=at.recup_route_split(v['voie'],v['codevoie'], lignes_adj2)
                    lignes_a_tester+=liste_rte_separe        
            lignes_a_tester=[x for x in lignes_a_tester if x not in liste_troncon_finale]
    liste_troncon_finale=list(set(liste_troncon_finale))
    return liste_troncon_finale

# 3. Boucler sur 'ensembl des lignes
On va faire tourner les traitements sur l'ensemble des lignes, en partant de la liste des id_ign, pour chaque ligne : 
- si la ligne ne remplit pas les critères d'exclusion on continu, sinon on passe à la suivante
- fonction de recherche des lignes associées
- df des association
- filtre de la liste des id_ign de base

In [422]:
# liste des id_ign
liste_id_ign_base=np.array(df_avec_rd_pt.id_ign.tolist())

In [423]:
len(liste_id_ign_base)

12680

In [424]:
dico_fin={}
lignes_traitees=np.array([],dtype='<U24')
for i,l in enumerate(liste_id_ign_base) :
    if len(lignes_traitees)>=len(liste_id_ign_base) : break
    if l in lignes_traitees : 
        continue
    if i%100==0 : 
        print(i, datetime.now(), f'nb lignes traitees : {len(lignes_traitees)}')
    #critère d'exclusion : si la ligne est une bretelle ou si la ligne à un nb de lignes qui intersecte debut et fin >2, ou si c'est un rd pt
    ligne=df_avec_rd_pt.set_index('id_ign').loc[l]
    if ((ligne['nb_intrsct_src'] >2 and ligne['nb_intrsct_tgt']) > 2) or ligne['nature']=='Bretelle' or ~np.isnan(ligne['id_rdpt']):
        continue
    else : 
        liste_ligne=lignes_troncon_elem(df_avec_rd_pt, l)  
        lignes_traitees=np.unique(np.append(lignes_traitees,liste_ligne))
    for ligne in liste_ligne : 
        dico_fin[ligne]=i
    

0 2019-09-03 16:31:33.870873 nb lignes traitees : 0
100 2019-09-03 16:31:41.119197 nb lignes traitees : 350
200 2019-09-03 16:31:48.898696 nb lignes traitees : 714
300 2019-09-03 16:31:55.897135 nb lignes traitees : 1084
400 2019-09-03 16:32:03.332871 nb lignes traitees : 1443
600 2019-09-03 16:32:17.829559 nb lignes traitees : 2194
700 2019-09-03 16:32:25.093522 nb lignes traitees : 2610
800 2019-09-03 16:32:31.935703 nb lignes traitees : 2948
900 2019-09-03 16:32:37.746870 nb lignes traitees : 3163
1000 2019-09-03 16:32:43.464339 nb lignes traitees : 3464
1100 2019-09-03 16:32:48.744346 nb lignes traitees : 3704
1300 2019-09-03 16:32:59.335679 nb lignes traitees : 4154
1400 2019-09-03 16:33:03.897114 nb lignes traitees : 4374
1600 2019-09-03 16:33:12.738839 nb lignes traitees : 4686
1800 2019-09-03 16:33:21.221273 nb lignes traitees : 4998
1900 2019-09-03 16:33:26.438825 nb lignes traitees : 5216
2400 2019-09-03 16:33:47.918263 nb lignes traitees : 6089
2500 2019-09-03 16:33:51.52680

In [425]:
df_test=pd.DataFrame.from_dict(dico_fin, orient='index').reset_index()
df_test.columns=['id','idtronc']
df_test.loc[df_test.duplicated('id')]

,id,idtronc


In [427]:
df_avec_rd_pt.merge(df_test, left_on='id_ign',right_on='id')[['id_ign','numero','codevoie_d','geom','idtronc']].to_file(r'R:\OTV\test_linearisation\test_1.shp')

In [26]:
%%time
a=np.array([i for i in range(100000)])
c=[]
for i,b in enumerate(a) :
    if b not in a :
        continue
    c=[b,b+1]
    a=a[np.isin(a,c, invert=True)]
    if i%50000==0 : print(i)

{'TRONROUT0000000339411580',
 'TRONROUT0000000339411600',
 'TRONROUT0000000339411601',
 'TRONROUT0000000339411606',
 'TRONROUT0000000339411607',
 'TRONROUT0000000339411608',
 'TRONROUT0000000339411609',
 'TRONROUT0000000339411610',
 'TRONROUT0000000339411611'}

# SUITE

In [114]:
toto=list(at.recup_troncon_elementaire('TRONROUT0000000032862365',df,[]))

In [115]:
toto.append('TRONROUT0000000032862365')

In [ ]:
toto

In [117]:
at.recup_troncon_parallele_v2(df,toto)

C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees\agrege_troncon.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))


'TRONROUT0000000032876496'

In [ ]:
for i in toto :
    tata=at.affecter_troncon_ligne(i,df,df2_chaussees)
    print(len(tata))

In [12]:
dico=at.affecter_troncon(df,df2_chaussees)

0eme occurence : TRONROUT0000000084053027 à 18:23:53 nb ligne traite : 0, nb ligne differente=0
indexError ligne  TRONROUT0000000032954220
indexError ligne  TRONROUT0000000033016282
indexError ligne  TRONROUT0000000249471326
indexError ligne  TRONROUT0000000336640149
indexError ligne  TRONROUT0000000224966440
indexError ligne  TRONROUT0000000032833606
indexError ligne  TRONROUT0000000032895598
indexError ligne  TRONROUT0000000032991823
indexError ligne  TRONROUT0000000223846575
indexError ligne  TRONROUT0000000255628539
indexError ligne  TRONROUT0000000032895522
indexError ligne  TRONROUT0000000032843156
indexError ligne  TRONROUT0000000032966066
indexError ligne  TRONROUT0000000129676068
indexError ligne  TRONROUT0000000032855812
indexError ligne  TRONROUT0000000033029153
indexError ligne  TRONROUT0000000323460798
indexError ligne  TRONROUT0000000032937456
indexError ligne  TRONROUT0000000089679809
indexError ligne  TRONROUT0000000211249489
5000eme occurence : TRONROUT0000000032859682

In [13]:
with ct.ConnexionBdd('local_otv') as c :
    at.inserer_dico(c, dico)

POUR TRAITEMENT DES VOIES A 2 CHAUSSES : TROUVER PARRALLELE

In [87]:
#liste des troncons (parametre entrant)
liste_troncon=toto

In [88]:
#conversion en geodf
gdf_lignes=gp.GeoDataFrame(df.loc[liste_troncon], geometry='geom')
gdf_lignes.to_file(r'D:\temp\otv\17\2_chaussees\lignes_a_gouper.shp')

In [89]:
#union des geometries
gdf_lignes2=linemerge(gdf_lignes.unary_union)

In [ ]:
gdf_lignes2.wkt

In [90]:
#centroid de la ligne
xmin,ymin,xmax,ymax=gdf_lignes2.interpolate(0.5, normalized=True).buffer(50).bounds
point=gdf_lignes2.interpolate(0.5, normalized=True)

In [91]:
point.wkt

'POINT (385035.1182861876 6561685.156661048)'

In [92]:
#recherche des lignes proches du centroid
lignes_possibles=df.cx[xmin:xmax,ymin:ymax]
lignes_possibles.to_file(r'D:\temp\otv\17\2_chaussees\lignes_possibles.shp')

In [ ]:
lignes_possibles

In [94]:
#uniquement les lignes non présentes dans la liste de troncons avec le même nom de voie
ligne_filtres=lignes_possibles.loc[(~lignes_possibles.index.isin(liste_troncon)) & (lignes_possibles['numero']==gdf_lignes.iloc[0]['numero'])]

In [95]:
#obtenir les distances au centroid
ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
#garder uniquement la valeur la plus proche du centroid
ligne_proche=ligne_filtres.loc[ligne_filtres['distance']==ligne_filtres['distance'].min()].index.tolist()[0]
ligne_proche

'TRONROUT0000000129676313'

POUR TRAITEMENT DES ROND POINTS

In [4]:
#polygoniser les lignes
#creer la gdf
df.crs={'init':'epsg:2154'}

In [15]:
#creer la lsite des rd points selon le critere atmo aura ; creer une gdf avec la liste
liste_rd_points=[t.buffer(0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14] # car un cercle à un rappor de ce type entre 12 et 13
dico_rd_pt=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point=gp.GeoDataFrame(dico_rd_pt, geometry=liste_rd_points)
gdf_rd_point.crs={'init':'epsg:2154'}
gdf_rd_point.columns=['id_rdpt', 'facteur','geometry']
#on créer aussi la meme donnees avec un buffer interiuer, pour ne garder que les lignes dans le buffer exteriuer et hors buffer interieur (cas de rond point enjmabeant uine 2*2 et prenant la 2*2 voie qui est dans le polygone
liste_rd_points_int=[t.buffer(-0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14]
dico_rd_pt_int=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point_int=gp.GeoDataFrame(dico_rd_pt_int, geometry=liste_rd_points_int)
gdf_rd_point_int.crs={'init':'epsg:2154'}
gdf_rd_point_int.columns=['id_rdpt', 'facteur','geometry']

In [63]:
#jointure spataile pour une gdf avec uniquement les lignes des rd_points avec le numéro
l_dans_p=gp.sjoin(df,gdf_rd_point,op='within') 

In [64]:
#suprimer les doublons
l_dans_p_ss_doublon=l_dans_p.reset_index().rename(columns={'index':'id_ign'}).drop_duplicates('id_ign').set_index('id_ign')

In [66]:
l_dans_p_int=gp.sjoin(df,gdf_rd_point_int,op='within')

In [67]:
l_dans_p_final=l_dans_p_ss_doublon.loc[~l_dans_p_ss_doublon.index.isin(l_dans_p_int.index.tolist())]

In [68]:
#lignes qui touchent rd points
#1.ligne qui intersectent avec id_rdpt
l_intersct_rdpt=gp.sjoin(df,l_dans_p_final.drop('index_right', axis=1), how='inner',op='intersects')

In [69]:
#2.filtre de celle contenue dans le rd points 
l_intersct_rdpt=l_intersct_rdpt.loc[~l_intersct_rdpt.index.isin(l_dans_p_final.index.tolist())][['id_rdpt','numero_left','codevoie_d_left']]

In [70]:
#trouver le nb de voies qui intersectent chaque rd point et leur noms. renomer les colonnes
carac_rd_pt=(pd.concat([l_intersct_rdpt.groupby('id_rdpt').numero_left.nunique(),
    l_intersct_rdpt.groupby('id_rdpt')['numero_left'].apply(lambda x: ','.join(set(x))),
    l_intersct_rdpt.groupby('id_rdpt')['codevoie_d_left'].apply(lambda x: ','.join(set(x)))], axis=1))
carac_rd_pt.columns=['nb_rte_rdpt', 'nom_rte_rdpt','codevoie_rdpt']

In [ ]:
l_dans_p_final

In [297]:
#ajouter l'id_rdpt aux données
gdf_l_rd_point=pd.concat([df.drop('id_rdpt',axis=1),l_dans_p_final.loc[:,'id_rdpt']],axis=1, sort=False)

In [298]:
#mettre à jour la nature
gdf_l_rd_point['nature']=gdf_l_rd_point.apply(lambda x : 'Rd_pt' if x.id_rdpt>=0 else x['nature'], axis=1)

In [299]:
#ajouter les infos du rd point (nb voies différentes et nom)
gdf_avec_rd_pt=gdf_l_rd_point.merge(carac_rd_pt, how='left',left_on='id_rdpt', right_index=True)

In [320]:
def assigne_rdpt(id_rdpt,numero, liste_num, codevoie, liste_code) : 
    if id_rdpt >=0 :
        if (numero in liste_num) or (codevoie in liste_code) :
            return True
        else : 
            return False

In [321]:
gdf_avec_rd_pt['assigne_rdpt']=gdf_avec_rd_pt.apply(lambda x : assigne_rdpt(x['id_rdpt'],x['numero'], x['nom_rte_rdpt_y'], x['codevoie_d'],x['codevoie_rdpt']),axis=1)

In [ ]:
#gdf_avec_rd_pt.loc[gdf_avec_rd_pt.loc[:,'id_rdpt']==l_dans_p['id_rdpt']]
gdf_avec_rd_pt['id_rdpt']==l_dans_p['id_rdpt'].astype('float64')
#l_dans_p.loc[l_dans_p.loc[:,'id_rdpt']].sort_index().sort_index(axis=1)])

In [ ]:
#pour export et verif uniquement

#gdf_l_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test2.shp')
#gdf_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test1.shp')
#l_intersct_rdpt.to_file(r'E:\Boulot\otv\test_linearisation_auto\test3.shp')
df.to_file(r'E:\Boulot\otv\test_linearisation_auto\test7.shp')
#df.to_file(r'D:\temp\otv\17\test_rd_point\test5.shp')

In [91]:
','.join(set(['a','a','b','c']))

'b,c,a'

In [228]:
id_ign_ligne='TRONROUT0000000342862858'
df_ligne = df.loc[id_ign_ligne]
ligne_traite_troncon=[]
ligne_traite_troncon.append(id_ign_ligne)
liste_ligne_suivantes=[] 
liste_ligne_touchees=[]

In [ ]:
for key, value in {'nb_intrsct_src':['source', 'src_geom'],'nb_intrsct_tgt':['target', 'tgt_geom']}.items() :
    df_touches_source = df.loc[(~df.index.isin(ligne_traite_troncon)) & ((df['source'] == df_ligne[value[0]]) | (df['target'] == df_ligne[value[0]]))]
    df_rd_pt=df.loc[df['id_rdpt']==df_touches_source.iloc[0]['id_rdpt']]
    l_df_rd_pt=df.loc[df.index.isin(gp.sjoin(df,df_rd_pt, how='inner',op='intersects').index.unique().tolist())]
    l_df_rd_pt2=l_df_rd_pt.loc[~l_df_rd_pt.index.isin(df_rd_pt.index.tolist())].index.unique().tolist()
    l_df_rd_pt3=df.loc[df.index.isin(l_df_rd_pt2),'codevoie_d'].unique().tolist()
    print(l_df_rd_pt3)
    #liste_ligne_touchees+=df_touches_source.index.tolist()

In [233]:
df_touches_source['id_rdpt'].values.tolist()

[nan]

# TEST

In [138]:
%%time
a=np.array([i for i in range(100000)])
c=[]
for i,b in enumerate(a) :
    if b not in a :
        continue
    c=[b,b+1]
    a=a[np.isin(a,c, invert=True)]
    if i%50000==0 : print(i)

AttributeError: module 'numpy' has no attribute 'aray'

In [137]:
%%time
a=[i for i in range(100000)]
c=[]
for i,b in enumerate(a) :
    if b not in a :
        continue
    c=[b,b+1]
    a=[x for x in a if x not in c]
    if i%50000==0 : print(i)

0
50000
Wall time: 5min 14s


In [420]:
lignes_traitees=np.array([],dtype='<U24')
'1' in lignes_traitees

False

In [416]:
lignes_traitees.dtype

dtype('<U24')